### Import libraries

In [11]:
using CSV
using DataFrames
using Turing
using Plots
using StatsBase
using StatsFuns: logistic
using MLBase

### Read data

In [12]:
df = CSV.read("WVS.csv")
deletecols!(df, :Column1)
y = convert(Array, df[:y])
X = convert(Matrix, df[setdiff(names(df), [:y])]);
println(typeof(X))
println(typeof(y))
println(countmap(y))

Array{Float64,2}
Array{Int64,1}
Dict(2=>1862,3=>811,1=>2708)


### Define Ordered Logistic distribution

In [13]:
struct OrderedLogistic{T1, T2} <: DiscreteUnivariateDistribution
   η::T1
   cutpoints::Vector{T2}
end

function Distributions.logpdf(d::OrderedLogistic, k::Int)
   
    K = length(d.cutpoints)+1

    c =  d.cutpoints
    
    if k==1
        logp= log(logistic(c[k]-d.η))
    elseif k<K
        logp= log(logistic(c[k]-d.η) - logistic(c[k-1]-d.η))
    else
        logp= log(1-logistic(c[k-1]-d.η))
    end
    
    return(logp)
end

### Model specification

In [14]:
### Turing model
@model m(X, y) = begin

    D = size(X, 2)

    # priors
    sigma ~ TruncatedNormal(0,1,0,Inf)
    
    c1 ~ Normal(0, 20)
    log_diff_c ~ Normal(0, 2)
    c2 = c1 + exp(log_diff_c)
    c = [c1, c2]
    
    beta ~ MvNormal(zeros(D), sigma * ones(D))

    lp = X * beta

    # likelihood
    for i = 1:length(y)
        y[i] ~ OrderedLogistic(lp[i], c)
    end
end

m (generic function with 3 methods)

### Sampling

In [15]:
steps = 10000
chain = sample(m(X, y), NUTS(steps, 0.65));

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfiniteθ = true
│   isfiniter = false
│   isfiniteℓπ = false
│   isfiniteℓκ = false
└ @ AdvancedHMC /Users/kcft114/.julia/packages/AdvancedHMC/YWXfk/src/hamiltonian.jl:36
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfiniteθ = true
│   isfiniter = false
│   isfiniteℓπ = false
│   isfiniteℓκ = false
└ @ AdvancedHMC /Users/kcft114/.julia/packages/AdvancedHMC/YWXfk/src/hamiltonian.jl:36
┌ Info: Found initial step size
│   init_ϵ = 0.025
└ @ Turing.Inference /Users/kcft114/.julia/packages/Turing/3Ij4X/src/inference/hmc.jl:364
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfiniteθ = true
│   isfiniter = false
│   isfiniteℓπ = false
│   isfiniteℓκ = false
└ @ AdvancedHMC /Users/kcft114/.julia/packages/AdvancedHMC/YWXfk/src/hamiltonian.jl:36
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfiniteθ = true
│   is

### Parameter estimates

In [16]:
show(chain)

Object of type Chains, with data of type 9000×21×1 Array{Union{Missing, Float64},3}

Log evidence      = 0.0
Iterations        = 1:9000
Thinning interval = 1
Chains            = 1
Samples per chain = 9000
internals         = eval_num, lp, acceptance_rate, hamiltonian_energy, is_accept, log_density, n_steps, numerical_error, step_size, tree_depth
parameters        = beta[3], c1, log_diff_c, sigma, beta[4], beta[6], beta[5], beta[1], beta[2], beta[7], beta[8]

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean       │ std       │ naive_se    │ mcse        │ ess     │ r_hat    │
│     │ Symbol     │ Float64    │ Float64   │ Float64     │ Float64     │ Any     │ Any      │
├─────┼────────────┼────────────┼───────────┼─────────────┼─────────────┼─────────┼──────────┤
│ 1   │ beta[1]    │ -0.0736091 │ 0.326922  │ 0.00344606  │ 0.00814165  │ 1701.08 │ 1.00029  │
│ 2   │ beta[2]    │ 0.0958954  │ 0.326843  │ 0.00344522  │ 0.00826656  │ 1665.48 │ 1.00014  │
│ 3   │

In [17]:
e_log_diff_c = exp.(chain[:log_diff_c].value.data)[:,1,1]
c1_est = chain[:c1].value.data[:,1,1]
c2_est = c1_est + e_log_diff_c;
println(mean(c1_est))
println(mean(c2_est))

0.14852671664785083
1.9497495954146393


In [23]:
# bonus: plots of posterior distributions
histogram(c1_est , bar_width=0.04, legend=false, title="Posterior distributions in Julia / Turing", yaxis=nothing)
histogram!(c2_est )
savefig("posterior_plot.png" )

## How to make predictions now? Using the generative model.

In [27]:
function Distributions.rand(d::OrderedLogistic)
    cutpoints = d.cutpoints
    η = d.η  
    
    if !issorted(cutpoints)
        error("cutpoints are not sorted")
    end

    K = length(cutpoints)+1
    c = vcat(-Inf, cutpoints, Inf)
    l = [i for i in zip(c[1:(end-1)],c[2:end])]
    ps = [logistic(η - l[i][1]) - logistic(η - l[i][2]) for i in 1:K]
    k = findall(ps.== maximum(ps))[1]
    
    if all(ps.>0)
        return(k)
    else
        return(-Inf)
    end
end

In [28]:
beta_est = chain[:beta].value.data[:,:,1]';
lp_post = X * beta_est;

y_pred_samps = zeros(size(lp_post));

for i in 1:size(y_pred_samps,1)
    for j in 1:size(y_pred_samps,2)
        
        c1 = c1_est[j,1,1]
        c2 = c2_est[j,1,1]
        c = [c1, c2]
        
        dist = OrderedLogistic(lp_post[i,j], c)
        
        y_pred_samps[i,j] = rand(dist)
    end
end

In [29]:
y_pred = zeros(size(y_pred_samps, 1));

for i in 1:length(y_pred)
    
    p1 = mean(y_pred_samps[i,:] .== 1)
    p2 = mean(y_pred_samps[i,:] .== 2)
    p3 = mean(y_pred_samps[i,:] .== 3)
    probs = [p1, p2, p3]
    
    y_pred[i] = sum((probs .== maximum(probs)) .* [1, 2, 3])
end

y_pred = convert(Array{Int64,1}, y_pred);
countmap(y_pred)

Dict{Int64,Int64} with 2 entries:
  2 => 1312
  1 => 4069

### Accuracy, confusion matrix

In [30]:
mean(y .== y_pred)

0.47834974911726447

In [31]:
C = confusmat(3, y, y_pred)

3×3 Array{Int64,2}:
 2201  507  0
 1489  373  0
  379  432  0